In [1]:
from ufunctions import *
from functools import reduce
from pandas.tseries.offsets import *

In [2]:
dfQuote = GetQuote(ticker='SAVE', start='2018-01-01')

[*********************100%***********************]  1 of 1 completed


In [3]:
df = dfQuote.copy()
df['Adj Close'] = list(df["Adj Close"].pct_change())
df1 = RelativeStrengthIndex(dfQuote,14)
df2 = macdIndex(dfQuote,12,26,9)

# compile the list of dataframes you want to merge
list_df = [df,df1,df2]
df_final = pd.concat(list_df, axis=1)
df_final.dropna(inplace=True)
df_final.rename(columns={'Close': 'Target'}, inplace=True)
df_final.tail()

,Open,High,Low,Target,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,
2021-09-20,24.629999,25.070000,24.129999,25.059999,0.001999,4735800,49.92,-0.452311
2021-09-21,25.219999,25.540001,24.379999,24.639999,-0.016760,3848100,46.39,-0.412828
2021-09-22,24.780001,25.680000,24.700001,25.260000,0.025162,4030600,51.81,-0.369260
2021-09-23,25.500000,26.620001,25.379999,26.299999,0.041172,4724500,59.25,-0.307982
2021-09-24,26.270000,26.980000,26.250000,26.719999,0.015970,3485300,61.81,-0.231413


In [45]:
def prepTables(df_original, locShift = 3):

    df_original.reset_index(inplace=True)
    df_original = df_original.append(pd.DataFrame({'Date': pd.date_range(start=df_original.Date.iloc[-1], periods=locShift+1, freq=BDay(), closed='right')}))
    df_original.set_index(['Date'],inplace=True)
    df_original.loc[:, df_original.columns != 'Target'] = df_original.loc[:, df_original.columns != 'Target'].shift(locShift)
    df_target = df_original[['Target']]
    df_features = df_original.loc[:, df_original.columns != 'Target']
    df_back_test = df_target.merge(df_features, left_index=True, right_index=True).dropna()
    df_predict = df_original.tail(locShift)

    return df_predict,df_back_test
    

In [48]:
df_ = prepTables(df_final)[1]

In [49]:
df_

,Target,index,Open,High,Low,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,,
2018-01-26,41.480000,0.0,43.270000,44.669998,42.500000,0.029079,1902600.0,46.51,-0.079888
2018-01-29,41.419998,1.0,41.700001,42.860001,40.759998,-0.068625,3892400.0,37.87,-0.150418
2018-01-30,41.830002,2.0,41.389999,41.619999,39.529999,-0.032507,1956500.0,34.80,-0.268692
2018-01-31,42.119999,3.0,40.570000,41.490002,40.139999,0.032354,1506200.0,39.86,-0.389337
2018-02-01,42.020000,4.0,41.490002,41.740002,41.160000,-0.001447,878200.0,39.70,-0.505214
...,...,...,...,...,...,...,...,...,...
2021-09-20,25.059999,918.0,24.379999,24.830000,24.139999,0.012265,2885900.0,47.43,-0.588580
2021-09-21,24.639999,919.0,24.790001,25.280001,24.719999,0.000404,3103200.0,47.51,-0.545142
2021-09-22,25.260000,920.0,24.969999,25.549999,24.610001,0.009689,3823300.0,49.49,-0.499258


In [42]:
df_original = df_final.copy()
locShift = 3
df_original.reset_index(inplace=True)
df_original = df_original.append(pd.DataFrame({'Date': pd.date_range(start=df_original.Date.iloc[-1], periods=locShift+1, freq=BDay(), closed='right')}))
df_original.set_index(['Date'],inplace=True)
df_original.loc[:, df_original.columns != 'Target'] = df_original.loc[:, df_original.columns != 'Target'].shift(locShift)
df_target = df_original[['Target']]
df_features = df_original.loc[:, df_original.columns != 'Target']
df_back_test = df_target.merge(df_features, left_index=True, right_index=True).dropna()
df_predict = df_original.tail(locShift)

In [44]:
df_back_test

,Target,Open,High,Low,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,
2018-01-26,41.480000,43.270000,44.669998,42.500000,0.029079,1902600.0,46.51,-0.079888
2018-01-29,41.419998,41.700001,42.860001,40.759998,-0.068625,3892400.0,37.87,-0.150418
2018-01-30,41.830002,41.389999,41.619999,39.529999,-0.032507,1956500.0,34.80,-0.268692
2018-01-31,42.119999,40.570000,41.490002,40.139999,0.032354,1506200.0,39.86,-0.389337
2018-02-01,42.020000,41.490002,41.740002,41.160000,-0.001447,878200.0,39.70,-0.505214
...,...,...,...,...,...,...,...,...
2021-09-20,25.059999,24.379999,24.830000,24.139999,0.012265,2885900.0,47.43,-0.588580
2021-09-21,24.639999,24.790001,25.280001,24.719999,0.000404,3103200.0,47.51,-0.545142
2021-09-22,25.260000,24.969999,25.549999,24.610001,0.009689,3823300.0,49.49,-0.499258


In [41]:
df_original.tail()

,Open,High,Low,Target,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,
2021-09-23,24.629999,25.070000,24.129999,26.299999,0.001999,4735800.0,49.92,-0.452311
2021-09-24,25.219999,25.540001,24.379999,26.719999,-0.016760,3848100.0,46.39,-0.412828
2021-09-27,24.780001,25.680000,24.700001,NaN,0.025162,4030600.0,51.81,-0.369260
2021-09-28,25.500000,26.620001,25.379999,NaN,0.041172,4724500.0,59.25,-0.307982
2021-09-29,26.270000,26.980000,26.250000,NaN,0.015970,3485300.0,61.81,-0.231413


In [ ]:
df_features

In [16]:
df_target.merge(df_features, left_index=True, right_index=True)

,Target,Open,High,Low,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,
2018-01-23,44.590000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-24,41.529999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-25,40.180000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-26,41.480000,43.270000,44.669998,42.500000,0.029079,1902600.0,46.51,-0.079888
2018-01-29,41.419998,41.700001,42.860001,40.759998,-0.068625,3892400.0,37.87,-0.150418
...,...,...,...,...,...,...,...,...
2021-09-24,26.719999,24.780001,25.680000,24.700001,0.025162,4030600.0,51.81,-0.369260
2021-09-24,NaN,25.219999,25.540001,24.379999,-0.016760,3848100.0,46.39,-0.412828
2021-09-24,NaN,24.780001,25.680000,24.700001,0.025162,4030600.0,51.81,-0.369260


In [5]:
print(df_target.tail())
print(df_features.tail())

               Target
Date                 
2021-09-23  26.299999
2021-09-24  26.719999
2021-09-24        NaN
2021-09-27        NaN
2021-09-28        NaN
                 Open       High        Low  Adj Close     Volume  RSI14  \
Date                                                                       
2021-09-23  24.629999  25.070000  24.129999   0.001999  4735800.0  49.92   
2021-09-24  25.219999  25.540001  24.379999  -0.016760  3848100.0  46.39   
2021-09-24  24.780001  25.680000  24.700001   0.025162  4030600.0  51.81   
2021-09-27  25.500000  26.620001  25.379999   0.041172  4724500.0  59.25   
2021-09-28  26.270000  26.980000  26.250000   0.015970  3485300.0  61.81   

            SigMACD_12_26_9  
Date                         
2021-09-23        -0.452311  
2021-09-24        -0.412828  
2021-09-24        -0.369260  
2021-09-27        -0.307982  
2021-09-28        -0.231413  


In [20]:
df_original.tail()

,Open,High,Low,Target,Adj Close,Volume,RSI14,SigMACD_12_26_9
Date,,,,,,,,
2021-09-22,24.969999,25.549999,24.610001,25.260000,0.009689,3823300.0,49.49,-0.499258
2021-09-23,24.629999,25.070000,24.129999,26.299999,0.001999,4735800.0,49.92,-0.452311
2021-09-23,25.219999,25.540001,24.379999,NaN,-0.016760,3848100.0,46.39,-0.412828
2021-09-24,24.780001,25.680000,24.700001,NaN,0.025162,4030600.0,51.81,-0.369260
2021-09-27,25.500000,26.620001,25.379999,NaN,0.041172,4721800.0,59.25,-0.307982


In [ ]:
# def splitNlag(df):